In [1]:
import os
import numpy as np

from plotly.graph_objects import Figure
from plotly.graph_objects import Scatter

from  sxo.util.runtime.timeseries import RedisTs

In [2]:
os.environ["REDIS_PASS"] = "boss"


In [56]:
ts_bid = RedisTs('ts_bid_37374')
ts_ask = RedisTs('ts_ask_37374')
bid = ts_bid.get_range(name='bid')
ask = ts_ask.get_range(name='ask')
bid_ask = bid.join(ask, how='inner').reset_index()
bid_ask['t'] = bid_ask['t'].values.astype('datetime64[ms]')

df_plt = bid_ask.copy()[bid_ask.t > np.datetime64('2023-06-10 20:20:00')]


fig = Figure()
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['ask'], name='ask', mode="markers", marker={'size':4}))
fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['bid'], name='bid', mode="markers", marker={'size':4}))
fig.show()

In [70]:
import pandas as pd


def gen_binned_data(ticks:pd.DataFrame,
                    bin_size_sec:int,
                    time_col:str = "t",
                    bin_start_col:str = "bin_start",
                    bin_end_col:str = "bin_start"
                    ) -> pd.DataFrame:
    df = ticks.copy()

    def make_bin(x:pd.DataFrame) -> pd.DataFrame:
        return pd.Series({'a':123,'b':234})
    
    # pandas barfs if the multiplier of units is not unit
    df[bin_start_col] = df[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[s]')
    return df.groupby([bin_start_col]).apply(make_bin)
    

#nat = np.array([np.nan]).astype('datetime64')[0]
# bin_end_times = np.append(bin_start_times[1:],nat)
# bid_ask['bin_end'] = bin_end_times

gen_binned_data(bid_ask, 60)


,a,b
bin_start,,
2023-06-06 19:22:00,123,234
2023-06-06 19:23:00,123,234
2023-06-06 19:27:00,123,234
2023-06-06 19:29:00,123,234
2023-06-06 20:27:00,123,234
...,...,...
2023-06-12 19:04:00,123,234
2023-06-12 19:05:00,123,234
2023-06-12 19:06:00,123,234


In [30]:
bin_start_times = bid_ask['t'].values.astype('datetime64[10m]')
np.unique(bin_start_times)

array(['2023-06-06T19:20', '2023-06-06T20:20', '2023-06-06T20:30',
       '2023-06-06T20:40', '2023-06-06T20:50', '2023-06-08T19:30',
       '2023-06-08T19:40', '2023-06-08T19:50', '2023-06-08T20:00',
       '2023-06-08T20:10', '2023-06-12T18:10', '2023-06-12T18:20'],
      dtype='datetime64[10m]')